In [1]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
#-- Exploring the data ---#
print(len(documents))
print(documents[:5])

1103663
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [3]:
#--- Text Preprocessing ----- #

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

def lemmatize_stemming(text):
    stemmer=SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vsurampu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [4]:
#--Preprocess the headline text, saving the results as ‘processed_docs’--#
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [5]:
#-- Creating a dictionary from 'processed_docs' containing # of times word appears--#
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [6]:
##-- Filter out tokens which appear in less than 15 documents, more than 50% of corpus and keeping only 100000 frequent tokens--#
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [10]:
#-- Bag of Words--#

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [8]:
#--Preview for Sample Document --#
bow_corpus[4310]
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))


Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4014 ("dampen") appears 1 time.


In [11]:
#--TFIDF--#
    
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5892908867507543),
 (1, 0.38929654337861147),
 (2, 0.4964985175717023),
 (3, 0.5046520327464028)]


In [12]:
#-- Running LDA using Bag of Words -- #
    
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)


In [16]:
#-- Running LDA using TF-IDF --#

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.011*"live" + 0.010*"donald" + 0.008*"farmer" + 0.008*"price" + 0.007*"cattl" + 0.007*"michael" + 0.006*"wednesday" + 0.006*"mine" + 0.006*"dairi" + 0.006*"drought"
Topic: 1 Word: 0.014*"news" + 0.014*"market" + 0.010*"final" + 0.009*"share" + 0.008*"australian" + 0.008*"open" + 0.007*"world" + 0.007*"juli" + 0.006*"septemb" + 0.006*"hill"
Topic: 2 Word: 0.015*"charg" + 0.014*"murder" + 0.014*"court" + 0.010*"alleg" + 0.009*"jail" + 0.009*"polic" + 0.008*"accus" + 0.008*"child" + 0.008*"drug" + 0.008*"sentenc"
Topic: 3 Word: 0.014*"rural" + 0.011*"govern" + 0.008*"fund" + 0.007*"plan" + 0.007*"drum" + 0.007*"chang" + 0.007*"council" + 0.006*"health" + 0.006*"budget" + 0.005*"nation"
Topic: 4 Word: 0.017*"crash" + 0.015*"polic" + 0.011*"miss" + 0.011*"interview" + 0.011*"die" + 0.010*"shoot" + 0.009*"woman" + 0.009*"search" + 0.008*"fatal" + 0.008*"christma"
Topic: 5 Word: 0.013*"turnbul" + 0.009*"plead" + 0.008*"octob" + 0.008*"monday" + 0.008*"john" + 0.008*"malcolm" +

In [17]:
#-- Topic of our sample document 4310 --#
    
processed_docs[4310]

for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

#-- Peformance evaluation by classifying example document using LDA TF-IDF model--#

for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))
    


Score: 0.4203391373157501	 
Topic: 0.024*"south" + 0.024*"adelaid" + 0.023*"north" + 0.016*"donald" + 0.015*"west" + 0.015*"indigen" + 0.013*"coast" + 0.012*"abus" + 0.012*"citi" + 0.012*"flood"

Score: 0.22017015516757965	 
Topic: 0.033*"death" + 0.018*"turnbul" + 0.017*"price" + 0.017*"island" + 0.014*"rise" + 0.013*"forc" + 0.013*"royal" + 0.013*"claim" + 0.012*"drum" + 0.011*"commiss"

Score: 0.2193368822336197	 
Topic: 0.037*"trump" + 0.023*"australia" + 0.016*"win" + 0.014*"time" + 0.013*"gold" + 0.013*"take" + 0.012*"meet" + 0.011*"lead" + 0.010*"beat" + 0.010*"port"

Score: 0.020029688253998756	 
Topic: 0.024*"crash" + 0.023*"canberra" + 0.019*"hospit" + 0.014*"road" + 0.013*"die" + 0.012*"polit" + 0.012*"green" + 0.011*"public" + 0.011*"resid" + 0.010*"question"

Score: 0.020024804398417473	 
Topic: 0.037*"govern" + 0.020*"live" + 0.020*"famili" + 0.019*"miss" + 0.017*"power" + 0.015*"peopl" + 0.014*"life" + 0.014*"farmer" + 0.014*"feder" + 0.012*"week"

Score: 0.020022811368

In [18]:
#-- Exploring words in each topic --#

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))


Topic: 0 
Words: 0.023*"kill" + 0.017*"elect" + 0.017*"say" + 0.015*"attack" + 0.015*"state" + 0.014*"china" + 0.013*"children" + 0.013*"deal" + 0.011*"talk" + 0.010*"leader"
Topic: 1 
Words: 0.037*"trump" + 0.023*"australia" + 0.016*"win" + 0.014*"time" + 0.013*"gold" + 0.013*"take" + 0.012*"meet" + 0.011*"lead" + 0.010*"beat" + 0.010*"port"
Topic: 2 
Words: 0.024*"crash" + 0.023*"canberra" + 0.019*"hospit" + 0.014*"road" + 0.013*"die" + 0.012*"polit" + 0.012*"green" + 0.011*"public" + 0.011*"resid" + 0.010*"question"
Topic: 3 
Words: 0.019*"perth" + 0.017*"melbourn" + 0.015*"sydney" + 0.015*"open" + 0.015*"world" + 0.014*"year" + 0.014*"tasmanian" + 0.014*"tasmania" + 0.012*"record" + 0.012*"australian"
Topic: 4 
Words: 0.024*"warn" + 0.022*"test" + 0.014*"victoria" + 0.014*"driver" + 0.013*"news" + 0.012*"victorian" + 0.011*"street" + 0.011*"violenc" + 0.011*"liber" + 0.010*"aborigin"
Topic: 5 
Words: 0.049*"polic" + 0.030*"charg" + 0.028*"court" + 0.025*"queensland" + 0.022*"murder

In [19]:
#--Testing model on unseen document--#

unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))


Score: 0.28310033679008484	 Topic: 0.023*"kill" + 0.017*"elect" + 0.017*"say" + 0.015*"attack" + 0.015*"state"
Score: 0.2266373634338379	 Topic: 0.025*"say" + 0.019*"nation" + 0.017*"school" + 0.015*"countri" + 0.015*"chang"
Score: 0.20682455599308014	 Topic: 0.024*"south" + 0.024*"adelaid" + 0.023*"north" + 0.016*"donald" + 0.015*"west"
Score: 0.18335382640361786	 Topic: 0.024*"crash" + 0.023*"canberra" + 0.019*"hospit" + 0.014*"road" + 0.013*"die"
Score: 0.016681797802448273	 Topic: 0.037*"govern" + 0.020*"live" + 0.020*"famili" + 0.019*"miss" + 0.017*"power"
Score: 0.016681162640452385	 Topic: 0.033*"death" + 0.018*"turnbul" + 0.017*"price" + 0.017*"island" + 0.014*"rise"
Score: 0.016680501401424408	 Topic: 0.037*"trump" + 0.023*"australia" + 0.016*"win" + 0.014*"time" + 0.013*"gold"
Score: 0.016680138185620308	 Topic: 0.019*"perth" + 0.017*"melbourn" + 0.015*"sydney" + 0.015*"open" + 0.015*"world"
Score: 0.016680138185620308	 Topic: 0.024*"warn" + 0.022*"test" + 0.014*"victoria" + 